Install Jupyter

In [ ]:



%pip install jupyter 

In [ ]:
ImportError

In [ ]:
import sys
print(sys.executable) 

In [ ]:
%pip install pandas numpy scikit-learn shap matplotlib

%pip install shap


IMPORT LIBRARIES

In [ ]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import shap 
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset

data = pd.read_csv(r'C:\Users\Sakshi Agarwal\Desktop\CV D\Explainable-Machine-Learning-Approaches-for-Cardiovascular-Disease-Risk-Assessment\cardio_data_processed.csv',sep=',' ,header=0,engine='python')

In [ ]:

column_names = data.columns.tolist()
print("\nNames of all columns:")
print(column_names)

In [ ]:
with open(r'C:\Users\Sakshi Agarwal\Desktop\CV D\Explainable-Machine-Learning-Approaches-for-Cardiovascular-Disease-Risk-Assessment\cardio_data_processed.csv') as file:
    for _ in range(5): 
        print(file.readline())


In [ ]:

print("\nSummary of the dataset:")
print(data.info())
print(data)


DATA PREPROCESSING

In [ ]:

numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].median())

In [39]:
print("Original age mean:", data['age'].mean())
print("Original age std:", data['age'].std())


Original age mean: 2.3044066146196637e-16
Original age std: 1.0


In [40]:
if 'age' in data.columns:
    data['age'] = (data['age'] - data['age'].mean()) / data['age'].std()  

if 'gender' in data.columns:
    data = pd.get_dummies(data, columns=['gender'], drop_first=True)  

In [33]:
print(data.dtypes)

id                       int64
age                    float64
height                   int64
weight                 float64
ap_hi                    int64
ap_lo                    int64
cholesterol              int64
gluc                     int64
smoke                    int64
alco                     int64
active                   int64
cardio                   int64
bmi                    float64
bp_category             object
bp_category_encoded     object
gender_2                 uint8
dtype: object


In [34]:

numeric_data = data.select_dtypes(include=[np.number])


correlation_matrix = numeric_data.corr().abs()


In [35]:
upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

In [36]:

to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]


to_drop = [column for column in to_drop if column in data.columns]


data = data.drop(columns=to_drop)


print("Dropped columns due to high correlation:", to_drop)
print("\nRemaining columns after dropping:")
print(data.columns.tolist())


Dropped columns due to high correlation: []

Remaining columns after dropping:
['id', 'age', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio', 'bmi', 'bp_category', 'bp_category_encoded', 'gender_2']


In [38]:
print("Mean age:", data['age'].mean())
print("Std age:", data['age'].std())


Mean age: 2.3044066146196637e-16
Std age: 1.0


In [37]:
X = data.drop('cardio', axis=1)
y = data['cardio']





DATA SPLITTING INTO TRAIN AND TEST 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

MODEL TRAINING USING RANDOM FOREST

In [ ]:
print(X_train.isnull().sum())  
print(y_train.isnull().sum())  


In [ ]:
print(X_train.shape)  
print(y_train.shape)  
print(y_train.head())  


In [41]:
X_train = pd.get_dummies(X_train)  


In [42]:
print(X_train.columns.tolist())


['id', 'age', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'bmi', 'gender_2', 'bp_category_Elevated', 'bp_category_Hypertension Stage 1', 'bp_category_Hypertension Stage 2', 'bp_category_Normal', 'bp_category_encoded_Elevated', 'bp_category_encoded_Hypertension Stage 1', 'bp_category_encoded_Hypertension Stage 2', 'bp_category_encoded_Normal']


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)


In [ ]:
import joblib
joblib.dump(rf_model, "rf_model.pkl")


MODEL EVALUATION

In [ ]:
print("X_train columns:", X_train.columns)
print("X_test columns:", X_test.columns)


In [ ]:
print("Columns in X_train but not in X_test:", set(X_train.columns) - set(X_test.columns))
print("Columns in X_test but not in X_train:", set(X_test.columns) - set(X_train.columns))


In [ ]:
for col in X_train.columns:
    if col not in X_test.columns:
        X_test[col] = 0  # or X_test[col] = X_train[col].mean() if appropriate


In [ ]:
X_test = X_test[X_train.columns]


In [ ]:
y_pred = rf_model.predict(X_test)
print("\nModel Accuracy:", (accuracy_score(y_test, y_pred))*100)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
sampled_X_test = X_test.sample(n=100, random_state=42)
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test)

In [ ]:



print("Shape of shap_values (class 1):", shap_values[1].shape)
print("Shape of X_test:", sampled_X_test.shape)


shap_values_class_1 = shap_values[1]  


if shap_values_class_1.shape[0] == X_test.shape[1]:
    shap_values_class_1 = shap_values_class_1.T


assert shap_values_class_1.shape[1] == X_test.shape[1], \
    f"Mismatch: SHAP values shape {shap_values_class_1.shape} vs X_test shape {X_test.shape}"


shap.summary_plot(shap_values_class_1, sampled_X_test, plot_type="bar")


In [ ]:



def get_patient_details():
    new_patient = {
        'age': int(input("Enter Age: ")),
        'gender': int(input("Enter Gender (0=Female, 1=Male): ")),
        'height': int(input("Enter Height (cm): ")),
        'weight': int(input("Enter Weight (kg): ")),
        'ap_hi': int(input("Enter Systolic BP: ")),
        'ap_lo': int(input("Enter Diastolic BP: ")),
        'cholesterol': int(input("Enter Cholesterol (1-3): ")),
        'gluc': int(input("Enter Glucose (1-3): ")),
        'smoke': int(input("Do you Smoke? (0=No, 1=Yes): ")),
        'alco': int(input("Do you Consume Alcohol? (0=No, 1=Yes): ")),
        'active': int(input("Are you Physically Active? (0=No, 1=Yes): "))
    }
    return new_patient


def predict_cvd():
    new_patient = get_patient_details()
    new_patient_df = pd.DataFrame([new_patient])
    new_patient_df = new_patient_df.reindex(columns=X_train.columns, fill_value=0)
    prediction = rf_model.predict(new_patient_df)[0]
    result = "Prone to CVD (1)" if prediction == 1 else "No CVD Risk (0)"
    print(f"\nPrediction: {result}")


predict_cvd()
